In [108]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime

In [158]:
hc_umls = pd.read_table("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/8_new_training/7_non/herb/1_herbal_compound_positive_umlsID.tsv")

In [159]:
print len(hc_umls)
hc_umls.head(1)

647


,compID,compName,synonym,diseaseID,omim_disease_name,umls_id,umls_name,avg
0,141,Pyridoxal,3-HYDROXY-5-(hydroxymethyl)-2-methylisonicotin...,239350,"Hyperphosphatemia, Polyuria, And Seizures",C0085681,HYPERPHOSPHATAEMIA (Hyperphosphatemia),0.952639


In [160]:
# umls_name to multiple rows
s = hc_umls['umls_name'].str.split('\(').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'umls_name'
del hc_umls['umls_name']
hc_umls = hc_umls.join(s)
hc_umls.umls_name = hc_umls.umls_name.str.rstrip('\)')

In [161]:
hc_umls.index = range(len(hc_umls))

In [115]:
df = pd.DataFrame()
print len(hc_umls)
cnt = 0
print "start : " + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for index, row in hc_umls.iterrows():
    hc = row['compName']
    di = row['umls_name']
    addr = 'https://www.ncbi.nlm.nih.gov/pubmed?term=(' + hc +'%5BTitle%2FAbstract%5D)%20AND%20' + di + '%5BTitle%2FAbstract%5D'
    
    html = urllib.urlopen(addr)
    soup = BeautifulSoup(html)
    
    a = soup.find_all("h3",attrs={'class':'result_count left'})

    if len(a) == 0: # 딱 하나만 검색 됐을 때
        searched = 1
    else :
        searched = a[0].text.split(" ")[-1]
    
    temp_df = pd.DataFrame({'compID': row['compID'], 'compName' : hc,
                            'diseaseID':row['diseaseID'], 'omim_disease_name' : row['omim_disease_name'],
                            'umls_id' : row['umls_id'],'umls_name' : di, 
                            'searched_count' : searched, 'avg' : row['avg']}, index=[cnt])
    df = df.append(temp_df)
    
    cnt = cnt + 1
    if cnt % 100 ==0 :
        print "cnt : {} ".format(cnt) + datetime.now().strftime('%Y-%m-%d %H:%M:%S')
          
print "end : " + datetime.now().strftime('%Y-%m-%d %H:%M:%S')

1050
start : 2017-06-10 21:20:17
cnt : 1002017-06-10 21:23:32
cnt : 2002017-06-10 21:27:05
cnt : 3002017-06-10 21:30:20
cnt : 4002017-06-10 21:33:41
cnt : 5002017-06-10 21:36:53
cnt : 6002017-06-10 21:40:14
cnt : 7002017-06-10 21:43:28
cnt : 8002017-06-10 21:46:35
cnt : 9002017-06-10 21:49:42
cnt : 10002017-06-10 21:52:47
end : 2017-06-10 21:54:22


In [125]:
df = df.drop_duplicates()

In [179]:
df.head()

,avg,compID,compName,diseaseID,omim_disease_name,searched_count,umls_id,umls_name
0,0.952639,141,Pyridoxal,239350,"Hyperphosphatemia, Polyuria, And Seizures",1,C0085681,HYPERPHOSPHATAEMIA
1,0.952639,141,Pyridoxal,239350,"Hyperphosphatemia, Polyuria, And Seizures",3,C0085681,Hyperphosphatemia
2,0.952639,141,Pyridoxal,239350,"Hyperphosphatemia, Polyuria, And Seizures",137,C0036572,Seizures
3,0.952639,141,Pyridoxal,239350,"Hyperphosphatemia, Polyuria, And Seizures",0,C0032617,Polyuria
4,0.930444,1167,Rescinnamine,161900,"Renal Failure, Progressive, With Hypertension",12,C1963138,Hypertension


In [177]:
df['searched_count'] = [int(x) for x in df.searched_count]

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [181]:
print len(df), len(df.drop_duplicates(['compName', 'umls_id'])), len(df.drop_duplicates(['compName', 'umls_name']))

1020 572 925


In [183]:
df_gr = df.groupby(['compID','compName','diseaseID','umls_id']).max()

In [184]:
df_gr

avg  \
compID compName              diseaseID umls_id              
141    Pyridoxal             239350    C0032617  0.952639   
                                       C0036572  0.952639   
                                       C0085681  0.952639   
153    Eicosapentanoic acid  140600    C0029408  0.808611   
                             161900    C0020538  0.778500   
                                       C0020544  0.778500   
                                       C0035078  0.778500   
                                       C1963138  0.778500   
                                       C1963154  0.778500   
416    (S)-(-)-atropine      143465    C1263846  0.822583   
                             147530    C0020507  0.784667   
                                       C0030193  0.784667   
                                       C0596992  0.784667   
                                       C1962977  0.784667   
                                       C2364139  0.784667   
                             182830    C0029124  0.749083   
                                       C0037772  0.749083   
                                       C0497327  0.749083   
                             608088    C0010200  0.841444   
                                       C0442874  0.841444   
                                       C1961131  0.841444   
562    Atropine              137750    C0017612  0.789944   
                                       C1842028  0.789944   
                             137760    C0339573  0.660139   
                             143465    C1263846  0.754083   
                             147530    C0020507  0.759639   
                                       C0030193  0.759639   
                                       C0596992  0.759639   
                                       C1962977  0.759639   
                                       C2364139  0.759639   
...                                                   ...   
78876  Hexitol               104130    C1306341  0.719083   
                             147530    C0020507  0.680333   
                                       C0030193  0.680333   
                                       C0596992  0.680333   
                                       C1962977  0.680333   
                                       C2364139  0.680333   
                             181500    C0036341  0.766583   
                             239350    C0032617  0.760000   
                                       C0036572  0.760000   
                                       C0085681  0.760000   
                             604416    C0001144  0.714639   
                                       C0085652  0.714639   
                                       C0678108  0.714639   
                                       C0702166  0.714639   
                                       C1692886  0.714639   
83229  3-Cyanopropanoic acid 102300    C0035258  0.823278   
                                       C1876177  0.823278   
                             147530    C0020507  0.847361   
                                       C0030193  0.847361   
                                       C0596992  0.847361   
                                       C1962977  0.847361   
                                       C2364139  0.847361   
                             157300    C0338480  0.716389   
                             190300    C0393615  0.707417   
                                       C1860861  0.707417   
                             300125    C0149931  0.680750   
                             605021    C0037769  0.853389   
                                       C0917800  0.853389   
86025  Oxygen, atomic        230800    C0017205  0.595028   
                                       C1961835  0.595028   

                                                                                 omim_disease_name  \
compID compName              diseaseID umls_id                                                       
141   

In [185]:
a = df_gr.reset_index(level=['compID', 'compName', 'diseaseID', 'umls_id'])

In [193]:
a[(a.compName != 'Testosterone') & (a.searched_count >100)].compName.drop_duplicates()

1                      Pyridoxal
11              (S)-(-)-atropine
26                      Atropine
130                   Irinotecan
157         (-)-Prostaglandin E1
173          Aminolevulinic acid
288              (+)-Vinblastine
341                     Cortisol
402                    Cortisone
445                    Ephedrine
533    (+)-Prostaglandin F2alpha
556                   Salicylate
Name: compName, dtype: object

In [195]:
a[a.searched_count != 0].compName.drop_duplicates()

1                                              Pyridoxal
4                                   Eicosapentanoic acid
10                                      (S)-(-)-atropine
21                                              Atropine
46                                          Testosterone
126                                           Irinotecan
153                                 (-)-Prostaglandin E1
173                                  Aminolevulinic acid
178                                       Salicylic acid
195                                           Cinacalcet
197                                           Probenecid
207                                      Podophyllotoxin
220                                         Rescinnamine
231                                solifenacin succinate
250                                          Citric acid
261                                              Estriol
273                                chenodeoxycholic acid
287                            

In [206]:
a[a.compName == '(+)-Prostaglandin F2alpha']

,compID,compName,diseaseID,umls_id,avg,omim_disease_name,searched_count,umls_name
526,10086,(+)-Prostaglandin F2alpha,137750,C0017612,0.796917,"Glaucoma 1, Open Angle, A; Glc1A",20,"Glaucoma, Open-Angle"
527,10086,(+)-Prostaglandin F2alpha,137750,C1842028,0.796917,"Glaucoma 1, Open Angle, A; Glc1A",0,GLC1A
528,10086,(+)-Prostaglandin F2alpha,137760,C0339573,0.745000,"Glaucoma, Primary Open Angle; Poag",6,Primary open angle glaucoma
529,10086,(+)-Prostaglandin F2alpha,145500,C0085580,0.763028,"Hypertension, Essential",6,Essential Hypertension
530,10086,(+)-Prostaglandin F2alpha,146255,C0018784,0.799194,"Hypoparathyroidism, Sensorineural Deafness, An...",0,Sensorineural deafness
531,10086,(+)-Prostaglandin F2alpha,146255,C0020626,0.799194,"Hypoparathyroidism, Sensorineural Deafness, An...",0,Hypoparathyroidism
532,10086,(+)-Prostaglandin F2alpha,146255,C0022658,0.799194,"Hypoparathyroidism, Sensorineural Deafness, An...",10,Renal disease
533,10086,(+)-Prostaglandin F2alpha,161900,C0020538,0.847972,"Renal Failure, Progressive, With Hypertension",113,Hypertensive disease
534,10086,(+)-Prostaglandin F2alpha,161900,C0020544,0.847972,"Renal Failure, Progressive, With Hypertension",1,Renal hypertension
535,10086,(+)-Prostaglandin F2alpha,161900,C0035078,0.847972,"Renal Failure, Progressive, With Hypertension",8,Renal Failure


In [57]:
df_gr.reset_index(level=['compName', 'umls_id'], inplace=True)

In [ ]:
umls에 여러개의 umls name이 있을 경우, cnt가 큰 것을 기준으로 한다.

In [65]:
df_gr[(df_gr.compName == "Pyridoxal")&(df_gr.umls_id == 'C0085681')]

,compName,umls_id,searched_count,umls_name
433,Pyridoxal,C0085681,3,Hyperphosphatemia


In [66]:
a = pd.merge(df_gr, hc_umls)

In [69]:
a[(a.compName == "Pyridoxal")&(a.umls_id == 'C0085681')]

,compName,umls_id,searched_count,umls_name,compID,synonym,diseaseID,omim_disease_name,avg
474,Pyridoxal,C0085681,3,Hyperphosphatemia,141,3-HYDROXY-5-(hydroxymethyl)-2-methylisonicotin...,239350,"Hyperphosphatemia, Polyuria, And Seizures",0.952639


In [73]:
print len(a.drop_duplicates()), len(a), len(df_gr)

607 629 572


In [ ]:
html = urllib.urlopen('https://www.ncbi.nlm.nih.gov/pubmed?term=(Irinotecan%5BTitle%2FAbstract%5D)%20AND%20Leukemi2a%5BTitle%2FAbstract%5D')

In [ ]:
html = urllib.urlopen('https://www.ncbi.nlm.nih.gov/pubmed/?term=(13+December+2016%3B+doi%3A10.1038%2Ftpj.2016.76.%5BTitle%2FAbstract%5D)+AND+Leukemia%5BTitle%2FAbstract%5D')

In [ ]:
soup = BeautifulSoup(html)

In [ ]:
a = soup.find_all("h3",attrs={"class":'result_count left'})

In [ ]:
len(a)

In [ ]:
dr_name = 'Irinotecan'
di_name = 'Leukemi2a'
addr = 'https://www.ncbi.nlm.nih.gov/pubmed?term=(' + dr_name +'%5BTitle%2FAbstract%5D)%20AND%20' + di_name + '%5BTitle%2FAbstract%5D'

html = urllib.urlopen(addr)
soup = BeautifulSoup(html)

a = soup.find_all("h3",attrs={'class':'result_count left'})

if (len(a) == 0):
    print(1)
#     continue
print(a[0].text.split(" ")[-1])



In [ ]:
len(a)print(a[0].text.split(" ")[-1])

In [ ]:
print(a[0].text.split(" ")[-1])